In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn

In [2]:
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = d2l.load_data_jay_lyrics()

In [3]:
#init params
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               nd.zeros(num_hiddens, ctx=ctx))
    W_xi, W_hi, b_i = _three() #input gate params
    W_xf, W_hf, b_f = _three() #forget gate params
    W_xo, W_ho, b_o = _three() #output layer params
    W_xc, W_hc, b_c = _three() #memory cell layer params
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [ W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o,
            W_xc, W_hc, b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [4]:
def init_lstm_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),
           nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [5]:
def lstm(inputs, state, params):
    [ W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o,
            W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [8]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [ ]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 158.233007, time 0.49 sec
 - 分开 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 
 - 不分开 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 我想想你你你 
epoch 100, perplexity 40.018373, time 0.48 sec
 - 分开 我想想你 我不要这样你 不知不觉 你已了我不多 一场个觉 你后了 我不好好活 我不能 你不了 我不
 - 不分开 我不要这样 我知你 我不要 我不要这你 不知不觉 你知了觉 我不能这生你 一知后觉 你后了 我不好
epoch 150, perplexity 10.040297, time 0.49 sec
 - 分开我 甩不不觉 我跟了这节奏 后知后觉 后知了一个秋 后知后觉 我该好好生活 我知好好生活 不知后觉 
 - 不分开活 我不 你不很 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不
